In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
# View all columns
pd.set_option('display.max_columns', None)


In [2]:
cwd = Path.cwd()
cwd

PosixPath('/home/ivan/dev/projects/slack_culling')

In [3]:
# List Folders, which are the channels
data_dir = cwd / 'data'
subdirs = [d for d in data_dir.glob('*/') if d.is_dir()]

for subdir in subdirs:
    # print(subdir)
    pass

In [4]:
# Test on art channel directory
directory_path = cwd / 'data' / 'art'
rows = []

for json_file in directory_path.glob('*.json'):
    with open(json_file, 'r') as file:
        data = json.load(file)
        # Capture the parent directory name
        directory_name = json_file.parent.name
        if isinstance(data, list):  # If the JSON is a list
            for item in data:
                item['file_name'] = json_file.stem  # File name without extension
                item['directory_name'] = directory_name  # Directory name
                rows.append(item)
        else:  # If the JSON is a dictionary
            data['file_name'] = json_file.stem
            data['directory_name'] = directory_name
            rows.append(data)

# Normalize nested JSON data into a flat table
df = pd.json_normalize(rows)
df.head(3)

,user,type,ts,client_msg_id,text,team,user_team,source_team,thread_ts,reply_count,reply_users_count,latest_reply,reply_users,replies,is_locked,subscribed,attachments,blocks,reactions,file_name,directory_name,user_profile.avatar_hash,user_profile.image_72,user_profile.first_name,user_profile.real_name,user_profile.display_name,user_profile.team,user_profile.name,user_profile.is_restricted,user_profile.is_ultra_restricted,parent_user_id,files,upload,display_as_bot,edited.user,edited.ts,last_read,subtype,bot_id,root.user,root.type,root.ts,root.client_msg_id,root.text,root.team,root.thread_ts,root.reply_count,root.reply_users_count,root.latest_reply,root.reply_users,root.replies,root.is_locked,root.subscribed,root.last_read,root.blocks,inviter
0,UHK1N52Q6,message,1653143791.618159,a06b6210-42b7-4d03-baf6-ca40948db100,<https://twitter.com/Grizzlegutweed1/status/15...,TH7UMG0KT,TH7UMG0KT,TH7UMG0KT,1653143791.618159,1.0,1.0,1653143825.705209,[UHK1N52Q6],"[{'user': 'UHK1N52Q6', 'ts': '1653143825.70520...",False,False,[{'from_url': 'https://twitter.com/Grizzlegutw...,"[{'type': 'rich_text', 'block_id': 'rw+', 'ele...","[{'name': 'fire', 'users': ['UHEB1D3PA', 'UHHS...",2022-05-21,art,7da94e642c57,https://avatars.slack-edge.com/2019-05-18/6280...,Roth,Roth,Roth,TH7UMG0KT,rothgar2112,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,UHK1N52Q6,message,1653143825.705209,f66584f5-09a2-4c9b-953b-ee24b52086c2,This city was also the first stop for Jackie's...,TH7UMG0KT,TH7UMG0KT,TH7UMG0KT,1653143791.618159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'type': 'rich_text', 'block_id': 'W54CE', 'e...","[{'name': 'eyes', 'users': ['UHEB1D3PA', 'UHHS...",2022-05-21,art,7da94e642c57,https://avatars.slack-edge.com/2019-05-18/6280...,Roth,Roth,Roth,TH7UMG0KT,rothgar2112,False,False,UHK1N52Q6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,UHEB1D3PA,message,1653143874.397839,15525551-b617-4197-aea4-537d4062f517,I need to remember this channel,TH7UMG0KT,TH7UMG0KT,TH7UMG0KT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'type': 'rich_text', 'block_id': 'Oev8', 'el...",NaN,2022-05-21,art,f77a8e01eea8,https://avatars.slack-edge.com/2023-04-17/5118...,Al,Al,Alejandro,TH7UMG0KT,alejandro.thornton,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Now iterate over all channels
directory_path = cwd / 'data'
rows = []

# Iterate over each subdirectory in the directory_path
for subdir in directory_path.iterdir():
    if subdir.is_dir():  # Ensure it's a directory
        # Iterate over JSON files in the current subdirectory
        for json_file in subdir.glob('*.json'):
            with open(json_file, 'r') as file:
                data = json.load(file)
                # Use subdir.name to get the directory name
                if isinstance(data, list):
                    for item in data:
                        item['file_name'] = json_file.stem  # Add the file name
                        item['directory_name'] = subdir.name  # Add the directory name
                        rows.append(item)
                else:
                    data['file_name'] = json_file.stem
                    data['directory_name'] = subdir.name
                    rows.append(data)

# Normalize nested JSON data into a flat table
df = pd.json_normalize(rows)

In [34]:
# Save Raw File
save_dir = cwd / 'processed_json'

df.to_csv(f'{save_dir}/slack_snapshot_03_26_24.csv', index=False)

### Step 2: Get Relevant Columns and Pare it down

In [5]:
# Read from file instead of re-doing it over and over

cols_v1 = ['user_profile.real_name', 'directory_name',
           'ts','text', 'reply_count', 
           'reactions', 'replies', 'user_profile.name', 'user']
posts = pd.read_csv('processed_json/slack_snapshot_03_26_24')
p2 = posts[cols_v1]
p2.shape

/tmp/ipykernel_210278/1940761885.py:6: DtypeWarning: Columns (37,38,39,40,41,42,43,44,46,47,48,49,51,52,53,58,59,60,61,62,63,65,66,68,69,70,71,72,73,75,76,77,78,79,80,82,83,84,85,87,88,89,90,91,92,93,94,95,96,97,98,99,100,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,155,157,158,159,160,161,162,163,165,166,167,168,170,171,172,173,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,203,204,205,206,207,208,210,211,212,213,214,215,216,218,219,220,223,224,225,226,227,228,229,231,232,233,234,236,237,238,239,240,241,242,243,244,246,247,249,251,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268) have mixed types. Specify dtype option on import or set low_memory=False.
  posts = pd.read_csv('processed_json/slack_snapshot_03_26_24')


(844942, 9)

In [6]:
p2.head(5)

,user_profile.real_name,directory_name,ts,text,reply_count,reactions,replies,user_profile.name,user
0,NaN,animestream,1.653143e+09,NaN,NaN,"[{'name': 'joy', 'users': ['UHK1N52Q6', 'UHH0R...",NaN,NaN,UHEB1D3PA
1,NaN,animestream,1.697145e+09,NaN,NaN,NaN,NaN,NaN,UHEB1D3PA
2,Michael,animestream,1.697146e+09,Asuka would be on sushi :unamused:,NaN,NaN,NaN,prohibited_art,UHHK3HG48
3,Michael,animestream,1.697146e+09,Rei would have chicken,NaN,NaN,NaN,prohibited_art,UHHK3HG48
4,Michael,animestream,1.697146e+09,Misato beef,NaN,NaN,NaN,prohibited_art,UHHK3HG48


In [7]:
# convert ts to timestamp
p2['time'] = pd.to_datetime(p2['ts'], unit='s')
p2.drop('ts', axis=1, inplace=True)
# Rename and reorganize columns
p2 = p2.rename(columns={'user_profile.real_name':'user',
                        'directory_name':'channel',
                        'user_profile.name':'username',
                        'user':'user_id'})

/tmp/ipykernel_210278/3971780017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p2['time'] = pd.to_datetime(p2['ts'], unit='s')
/tmp/ipykernel_210278/3971780017.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p2.drop('ts', axis=1, inplace=True)


In [8]:
p2.head(5)

,user,channel,text,reply_count,reactions,replies,username,user_id,time
0,NaN,animestream,NaN,NaN,"[{'name': 'joy', 'users': ['UHK1N52Q6', 'UHH0R...",NaN,NaN,UHEB1D3PA,2022-05-21 14:15:00.163179008
1,NaN,animestream,NaN,NaN,NaN,NaN,NaN,UHEB1D3PA,2023-10-12 21:17:55.162359040
2,Michael,animestream,Asuka would be on sushi :unamused:,NaN,NaN,NaN,prohibited_art,UHHK3HG48,2023-10-12 21:19:32.306399232
3,Michael,animestream,Rei would have chicken,NaN,NaN,NaN,prohibited_art,UHHK3HG48,2023-10-12 21:19:59.283819008
4,Michael,animestream,Misato beef,NaN,NaN,NaN,prohibited_art,UHHK3HG48,2023-10-12 21:20:06.183479040


In [9]:
postdir = cwd / 'processed_json'
p2.to_csv(f'{postdir}/posts.csv', index=False)

### Step 3: Parse Users and Channels Data

In [10]:
def parse_users():
    pass
def parse_channels():
    pass

In [11]:
userfile = 'data/users.json'
channelfile = 'data/channels.json'

In [12]:
usercols = ['real_name', 'name', 'id', 'profile'] 
users = pd.read_json(userfile)
users
u2 = users.copy()[usercols]
u2

,real_name,name,id,profile
0,Stephen,stephen.leininger,UH81VL1TK,"{'title': '', 'phone': '', 'skype': '', 'real_..."
1,Derek,froglenzen,UHAC3TPCZ,"{'title': '', 'phone': '', 'skype': '', 'real_..."
2,Al,alejandro.thornton,UHEB1D3PA,"{'title': '', 'phone': '', 'skype': '', 'real_..."
3,Ivan,skantastico,UHH0R7TTP,"{'title': 'Praetor Exelcius', 'phone': '', 'sk..."
4,Will,william.west.lane,UHH7GF9S9,"{'title': '', 'phone': '', 'skype': '', 'real_..."
5,Michael,prohibited_art,UHHK3HG48,"{'title': '', 'phone': '', 'skype': '', 'real_..."
6,Ben,benskee33,UHHS66KU7,"{'title': '', 'phone': '', 'skype': '', 'real_..."
7,Roth,rothgar2112,UHK1N52Q6,"{'title': '', 'phone': '', 'skype': '', 'real_..."
8,Google Drive,googledrive,UKRTZJ649,"{'title': '', 'phone': '', 'skype': '', 'real_..."
9,scryfall,scryfall,UN85LHRNK,"{'title': '', 'phone': '', 'skype': '', 'real_..."


In [13]:
users.columns

Index(['id', 'team_id', 'name', 'deleted', 'color', 'real_name', 'tz',
       'tz_label', 'tz_offset', 'profile', 'is_admin', 'is_owner',
       'is_primary_owner', 'is_restricted', 'is_ultra_restricted', 'is_bot',
       'is_app_user', 'updated', 'is_email_confirmed',
       'who_can_share_contact_card', 'is_workflow_bot'],
      dtype='object')

### Channels

In [14]:
channels = pd.read_json(channelfile)
channel_cols = ['name', 'id', 'creator', 'members', 'purpose', 'pins', 'topic', 'is_archived', 'is_general']
chans = channels[channel_cols]
chans.head(4)

,name,id,creator,members,purpose,pins,topic,is_archived,is_general
0,bookstream,CHBC77PMX,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...",{'value': 'discussions and channeling of thoug...,"[{'id': '1555531733.250500', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False
1,stackingsats,CHG467KSM,UHH0R7TTP,"[UH81VL1TK, UHEB1D3PA, UHH0R7TTP, UHH7GF9S9, U...","{'value': 'Stacking sats', 'creator': 'UHH0R7T...","[{'id': '1555032808.005500', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False
2,annoucements,CHGKGAJJY,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...",{'value': 'This channel is for workspace-wide ...,"[{'id': '1554065897.002200', 'type': 'C', 'cre...","{'value': 'Schedule and Info', 'creator': 'UHH...",False,True
3,shitposting,CHHPRTHGF,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}","[{'id': '1555442461.358700', 'type': 'C', 'cre...","{'value': 'autism, masturbation, hurricanes an...",False,False


In [15]:
# Get first post from each channel to get creation time
posts_sorted = p2.sort_values('time')
channel_created = posts_sorted.drop_duplicates(subset='channel', keep='first')
channel_created.head(5)

,user,channel,text,reply_count,reactions,replies,username,user_id,time
614928,NaN,annoucements,<@UHH0R7TTP> has joined the channel,NaN,NaN,NaN,NaN,UHH0R7TTP,2019-03-31 19:01:29.000199936
771974,NaN,3kp,<@UHH0R7TTP> has joined the channel,NaN,NaN,NaN,NaN,UHH0R7TTP,2019-03-31 19:01:30.000199936
306902,NaN,shitposting,<@UHH0R7TTP> has joined the channel,NaN,NaN,NaN,NaN,UHH0R7TTP,2019-03-31 21:05:13.000199936
105301,NaN,episodeguide,<@UHH0R7TTP> has joined the channel,NaN,NaN,NaN,NaN,UHH0R7TTP,2019-04-01 17:38:59.000199936
484460,NaN,culturewarz,<@UHH0R7TTP> has joined the channel,NaN,NaN,NaN,NaN,UHH0R7TTP,2019-04-03 03:20:56.000199936


In [18]:
# get channel and time created
chantime = channel_created[['channel', 'time']]
chantime = chantime.rename(columns={'time':'time_created'})
chantime

,channel,time_created
614928,annoucements,2019-03-31 19:01:29.000199936
771974,3kp,2019-03-31 19:01:30.000199936
306902,shitposting,2019-03-31 21:05:13.000199936
105301,episodeguide,2019-04-01 17:38:59.000199936
484460,culturewarz,2019-04-03 03:20:56.000199936
...,...,...
736358,basketball,2024-01-24 16:27:28.651019008
387485,lotr,2024-02-19 14:39:11.854758912
688932,theater,2024-02-19 16:52:10.850179072
313980,this-fuckin-guy,2024-02-22 06:58:57.076098816


In [19]:
# merge to channels and get delta for 'channel age'
c2 = chans.copy()
c3 = pd.merge(c2, chantime, left_on='name', right_on='channel', how='left')
c3

,name,id,creator,members,purpose,pins,topic,is_archived,is_general,channel,time_created
0,bookstream,CHBC77PMX,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...",{'value': 'discussions and channeling of thoug...,"[{'id': '1555531733.250500', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,bookstream,2019-04-03 03:27:49.000199936
1,stackingsats,CHG467KSM,UHH0R7TTP,"[UH81VL1TK, UHEB1D3PA, UHH0R7TTP, UHH7GF9S9, U...","{'value': 'Stacking sats', 'creator': 'UHH0R7T...","[{'id': '1555032808.005500', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,stackingsats,2019-04-07 16:39:42.000199936
2,annoucements,CHGKGAJJY,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...",{'value': 'This channel is for workspace-wide ...,"[{'id': '1554065897.002200', 'type': 'C', 'cre...","{'value': 'Schedule and Info', 'creator': 'UHH...",False,True,annoucements,2019-03-31 19:01:29.000199936
3,shitposting,CHHPRTHGF,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}","[{'id': '1555442461.358700', 'type': 'C', 'cre...","{'value': 'autism, masturbation, hurricanes an...",False,False,shitposting,2019-03-31 21:05:13.000199936
4,3kp,CHJ3UTLMU,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': 'Updates and Timeline', 'creator': '...","[{'id': '1575338162.468200', 'type': 'C', 'cre...",{'value': 'Talk about the TV SHOW here. Fellas...,False,False,3kp,2019-03-31 19:01:30.000199936
...,...,...,...,...,...,...,...,...,...,...,...
231,hustle-craft,C06E77PFRLZ,UHH7GF9S9,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}",NaN,"{'value': '', 'creator': '', 'last_set': 0}",False,False,hustle-craft,2024-01-14 18:41:04.062339072
232,basketball,C06FE7KMAMR,UHEB1D3PA,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH7GF9S9, U...",{'value': 'A channel where we talk about the s...,NaN,"{'value': 'Basketball', 'creator': 'UHEB1D3PA'...",False,False,basketball,2024-01-24 16:27:28.651019008
233,theater,C06K458MTFZ,UHAC3TPCZ,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}",NaN,"{'value': '', 'creator': '', 'last_set': 0}",False,False,theater,2024-02-19 16:52:10.850179072
234,lotr,C06KF15A2DB,UHAC3TPCZ,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': 'beat you', 'creator': 'UHAC3TPCZ', ...","[{'id': '1708353745.957669', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,lotr,2024-02-19 14:39:11.854758912


In [20]:
now = datetime.now()
def calculate_age(past_date):
    delta = relativedelta(now, past_date)
    return f"{delta.years} years, {delta.months} months, {delta.days} days"

# Apply the function to create the 'age' column
c3['age'] = c3['time_created'].apply(calculate_age)
c3

,name,id,creator,members,purpose,pins,topic,is_archived,is_general,channel,time_created,age
0,bookstream,CHBC77PMX,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...",{'value': 'discussions and channeling of thoug...,"[{'id': '1555531733.250500', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,bookstream,2019-04-03 03:27:49.000199936,"4 years, 11 months, 26 days"
1,stackingsats,CHG467KSM,UHH0R7TTP,"[UH81VL1TK, UHEB1D3PA, UHH0R7TTP, UHH7GF9S9, U...","{'value': 'Stacking sats', 'creator': 'UHH0R7T...","[{'id': '1555032808.005500', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,stackingsats,2019-04-07 16:39:42.000199936,"4 years, 11 months, 21 days"
2,annoucements,CHGKGAJJY,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...",{'value': 'This channel is for workspace-wide ...,"[{'id': '1554065897.002200', 'type': 'C', 'cre...","{'value': 'Schedule and Info', 'creator': 'UHH...",False,True,annoucements,2019-03-31 19:01:29.000199936,"4 years, 11 months, 28 days"
3,shitposting,CHHPRTHGF,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}","[{'id': '1555442461.358700', 'type': 'C', 'cre...","{'value': 'autism, masturbation, hurricanes an...",False,False,shitposting,2019-03-31 21:05:13.000199936,"4 years, 11 months, 28 days"
4,3kp,CHJ3UTLMU,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': 'Updates and Timeline', 'creator': '...","[{'id': '1575338162.468200', 'type': 'C', 'cre...",{'value': 'Talk about the TV SHOW here. Fellas...,False,False,3kp,2019-03-31 19:01:30.000199936,"4 years, 11 months, 28 days"
...,...,...,...,...,...,...,...,...,...,...,...,...
231,hustle-craft,C06E77PFRLZ,UHH7GF9S9,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}",NaN,"{'value': '', 'creator': '', 'last_set': 0}",False,False,hustle-craft,2024-01-14 18:41:04.062339072,"0 years, 2 months, 14 days"
232,basketball,C06FE7KMAMR,UHEB1D3PA,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH7GF9S9, U...",{'value': 'A channel where we talk about the s...,NaN,"{'value': 'Basketball', 'creator': 'UHEB1D3PA'...",False,False,basketball,2024-01-24 16:27:28.651019008,"0 years, 2 months, 4 days"
233,theater,C06K458MTFZ,UHAC3TPCZ,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}",NaN,"{'value': '', 'creator': '', 'last_set': 0}",False,False,theater,2024-02-19 16:52:10.850179072,"0 years, 1 months, 9 days"
234,lotr,C06KF15A2DB,UHAC3TPCZ,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': 'beat you', 'creator': 'UHAC3TPCZ', ...","[{'id': '1708353745.957669', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,lotr,2024-02-19 14:39:11.854758912,"0 years, 1 months, 10 days"
